In [53]:
import pandas as pd
from sklearn import preprocessing

In [108]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OKMeansEstimator
from h2o.estimators import H2ORandomForestEstimator
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,53 mins 18 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_mimivelloso_ugzxy4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.612 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [161]:
# Import a sample binary outcome train/test set into H2O
train =  pd.read_csv("output/trainclean.csv", index_col=0)
test = pd.read_csv("output/testclean.csv", index_col=0)

In [162]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [182]:
# Identify predictors and response
columnas = [a for a in h2train.columns if a != "price"]
predictors = columnas
response = "price"

In [178]:
train, valid = h2train.split_frame(ratios = [0.8], seed = 1234)

In [179]:
pros_gbm = H2OGradientBoostingEstimator(nfolds=10,
                                        seed=1111,
                                        keep_cross_validation_predictions = True)

In [186]:
pros_gbm.train(x=predictors,y=response,training_frame=train)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [187]:
perf = pros_gbm.model_performance()

In [196]:
perf


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 332176.8020242763
RMSE: 576.3478134115512
MAE: 317.86942580342753
RMSLE: 0.12737030262516116
Mean Residual Deviance: 332176.8020242763


In [197]:
pred = pros_gbm.predict(h2test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [200]:
test["price"] = pred.as_data_frame()
columnasentrega = ["id","price"]
entrega = test["price"]
test = test[columnasentrega]
test

,id,price
0,0,15569.944202
1,1,14984.745988
2,2,1146.312265
3,3,532.390367
4,4,3673.370957
...,...,...
13480,13480,2366.010803
13481,13481,4914.241716
13482,13482,2251.089420
13483,13483,1127.190413


In [201]:
test.to_csv("output/test5.csv", index= False)